In [6]:
import os
import cv2
import numpy as np

input_dir = './tmp/masks'
output_dir = './tmp/labels'

# Define class RGB colors
class_colors = {
    'Correct': (148, 104, 85),
    'Wrong': (121, 88, 224),
}

# Map class names to numeric labels
class_labels = {
    'Correct': 0,
    'Wrong': 1,
}

def mask_to_polygons(mask, class_label):
    # Find contours
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Convert contours to polygons
    H, W = mask.shape
    polygons = []
    for cnt in contours:
        if cv2.contourArea(cnt) > 200:  # Filter out small contours
            polygon = []
            for point in cnt:
                x, y = point[0]
                polygon.append(x / W)
                polygon.append(y / H)
            polygons.append((class_label, polygon))
    return polygons

def convert_mask_to_labels(input_dir, output_dir, class_colors, class_labels):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    for filename in os.listdir(input_dir):
        image_path = os.path.join(input_dir, filename)
        mask = cv2.imread(image_path)
        
        all_polygons = []
        for class_name, color in class_colors.items():
            class_label = class_labels[class_name]
            
            # Create binary mask for each class
            binary_mask = cv2.inRange(mask, color, color)
            
            # Get polygons for this class
            polygons = mask_to_polygons(binary_mask, class_label)
            all_polygons.extend(polygons)
        
        # Write to label file
        label_filename = os.path.splitext(filename)[0] + '.txt'
        output_path = os.path.join(output_dir, label_filename)
        with open(output_path, 'w') as f:
            for class_label, polygon in all_polygons:
                f.write(f"{class_label} ")
                for p_, p in enumerate(polygon):
                    if p_ == len(polygon) - 1:
                        f.write(f"{p}\n")
                    else:
                        f.write(f"{p} ")

# Run the conversion
convert_mask_to_labels(input_dir, output_dir, class_colors, class_labels)
